In [ ]:
##################################################################################
# [NOTE] train test
##################################################################################
# NEWER
# cd /lustre/groups/itg/teams/zeggini/projects/fungen-oa/analyses/olink/classifier/ukbb_3000_oatraits/
# srun --mem=100G -c 10 -p cpu_p   --qos=cpu_preemptible   --time 2-0:0:0 --pty singularity shell -B /home/itg/peter.kreitmaier/container.home.mqq2:/home/itg/peter.kreitmaier/ -B /lustre/groups/itg/ /lustre/groups/itg/shared/containers/worker_3.1.2
# TMPDIR=/lustre/groups/itg/teams/zeggini/users/peter.kreitmaier/tmp R
##################################################################################

In [ ]:
##################################################################################
# output directories will be:
# /lustre/groups/itg/teams/zeggini/projects/fungen-oa/analyses/olink/classifier/ukbb_3000_oatraits/Spine/5_all_strict_noagesex_more2until5INCID_superc01exclno/models_nrfeat8/models_benchmark_oleg_V1/
# /lustre/groups/itg/teams/zeggini/projects/fungen-oa/analyses/olink/classifier/ukbb_3000_oatraits/Spine/5_all_strict_noagesex_more2until5INCID_superc01exclno/models_nrfeat8/models_comb_benchmark_oleg_V1/
# /lustre/groups/itg/teams/zeggini/projects/fungen-oa/analyses/olink/classifier/ukbb_3000_oatraits/Spine/5_all_strict_noagesex_more2until5INCID_superc01exclno/models_sexagebmi_benchmark_oleg_V1
##################################################################################
start_time <- Sys.time()
##################################################################################
setwd("/lustre/groups/itg/teams/zeggini/projects/fungen-oa/analyses/olink/classifier/")
##################################################################################
require("caret")
require("randomForest")
require("limma")
require("data.table")
require("pROC")
require("xgboost")
require("plyr")
require("glmnet")
require("doParallel") 
require("ranger")
cl <- 5
registerDoParallel(cl)
##################################################################################
source("/lustre/groups/itg/teams/zeggini/projects/fungen-oa/analyses/olink/classifier/scripts/1_train_agnostic_0_utils.R")
##################################################################################
# define paths
PATH_OUT_INIT = "/lustre/groups/itg/teams/zeggini/projects/fungen-oa/analyses/olink/classifier/ukbb_3000_oatraits/"
######################################################
# CHANGE PARAMETERS HERE

#define input and output path
OA_PHENO_VEC = c("Spine")
NR_PROTEINS_VEC = c(8)

length(OA_PHENO_VEC) == length(NR_PROTEINS_VEC)
######################################################
IN_DIR_LABEL <- "more2until5INCID_superc01exclno"
# more2INCID_superc01exclno
######################################################
SUB_DIR = paste0("5_all_strict_noagesex_", IN_DIR_LABEL)

# IN_RELCOL_SUB <- "AOD_diff_case_t0_incid_more2until5","AOD_diff_case_t0_incid_more2"
IN_RELCOL_SUB <- "AOD_diff_case_t0_incid_more2until5"

PERFORM_TRAIN_LIST = list(prot_only = T, sexagebmi = T, comb = T)
PRED_MODE_VEC = c("prot_only", "comb", "sexagebmi")

PRED_COVS = c("sex", "bmi", "age")
PRED_COVS_NOTINCLUDE = setdiff(c("sex", "age", "bmi"), PRED_COVS)
######################################################
adjust_comb_nrprots = T

PERFORM_RFE = T

TRAIN_FI_RF_GRID = T

TRAIN_FI_RF_NR_ITER = 5
TRAIN_FI_RF_NR_FOLD = 5

SAMPLING = "rose"

VAL_RF = T
VAL_GLMNET = T
VAL_XG = T

TUNELENGTH_RF = 10
TUNELENGTH_GLMNET = 80
TUNELENGTH_XG = 10

#PRED_MODE_VEC = c("prot_only", "comb", "sexagebmi")


# define label of outcome files
if(PERFORM_RFE){
    label_rfe = "rfe"
}else{
    label_rfe = "norfe"
}

PATH_ANNO = "/lustre/groups/itg/teams/zeggini/projects/GPPAD_omics_projekct/POINTOLINK/RefFiles/2941ProteinIDNamesPanels.txt"
PATH_CC = "/lustre/groups/itg/teams/zeggini/projects/fungen-oa/analyses/olink/classifier/ukbb_3000/0_oainfo/SRplusPC-AllInds-AllVisits-AllCC.ukb9979.v9.txt"
#PATH_SAMPLES_CC_FILE = "/lustre/groups/itg/teams/zeggini/projects/fungen-oa/analyses/olink/classifier/ukbb_3000_oatraits/IN_OA_ALL_DATES_diff_case_WITHCOMBINED_until2PREVINCID_V9.rda"
##################################################################################
# [ITER] iterate over phenotypes and nr of proteins
# curr_elem = 1
sapply(1:length(OA_PHENO_VEC), FUN = function(curr_elem){

    OA_PHENO = OA_PHENO_VEC[curr_elem]
    NR_PROTEINS = NR_PROTEINS_VEC[curr_elem]

    # set_paths
    PATH_IN = paste0("/lustre/groups/itg/teams/zeggini/projects/fungen-oa/analyses/olink/classifier/ukbb_3000_oatraits/", "/",  OA_PHENO, "/", SUB_DIR,"/")
    PATH_IN_SUBDIR = paste0("models_nrfeat", NR_PROTEINS)

    PATH_IN_ORIG = PATH_IN
    PATH_IN = paste0(PATH_IN, PATH_IN_SUBDIR)


    # SET THESE DIRECOTIRES TO BENCHMARK FOLDERS
    # PATH_OUT_MODEL_PONLY = paste0(PATH_IN, "/models_benchmark_peter_V1/")
    # PATH_OUT_MODEL_COMB = paste0(PATH_IN, "/models_comb_benchmark_peter_V1/")
    # PATH_OUT_MODEL_sexagebmi = paste0(PATH_IN_ORIG, "/models_sexagebmi_benchmark_peter_V1/")

    PATH_OUT_MODEL_PONLY = paste0(PATH_IN, "/models_benchmark_oleg_V1/")
    PATH_OUT_MODEL_COMB = paste0(PATH_IN, "/models_comb_benchmark_oleg_V1/")
    PATH_OUT_MODEL_sexagebmi = paste0(PATH_IN_ORIG, "/models_sexagebmi_benchmark_oleg_V1/")

    PATH_OUT_TMPFILES_PONLY = paste0(PATH_OUT_MODEL_PONLY, "/tmp_files/")
    PATH_OUT_TMPFILES_COMB = paste0(PATH_OUT_MODEL_COMB, "/tmp_files/")
    PATH_OUT_TMPFILES_sexagebmi = paste0(PATH_OUT_MODEL_sexagebmi, "/tmp_files/")

    # create model and tmp_files (if not there)

    dir.create(PATH_IN) 

    dir.create(PATH_OUT_MODEL_PONLY) 
    dir.create(PATH_OUT_MODEL_COMB) 
    dir.create(PATH_OUT_MODEL_sexagebmi) 

    dir.create(PATH_OUT_TMPFILES_PONLY) 
    dir.create(PATH_OUT_TMPFILES_COMB) 
    dir.create(PATH_OUT_TMPFILES_sexagebmi) 

    p_anno = get_p_anno(in_p = PATH_ANNO)

    NR_PROTEINS_COMB = NR_PROTEINS + 1
    ##################################################################################
    # PRED_MODE = PRED_MODE_VEC[1]
    # prot_only|sexagebmi|comb
    ####################################################################################################################################################################
    sapply(PRED_MODE_VEC, FUN = function(PRED_MODE){
        ##################################################################################
        print("########################################################################################")
        print(PRED_MODE)
        print("########################################################################################")
        PERFORM_TRAIN = PERFORM_TRAIN_LIST[[PRED_MODE]]
        ##################################################################################
        # define current mode and tmpfile
        if(PRED_MODE == "prot_only"){
            PATH_OUT_MODEL = PATH_OUT_MODEL_PONLY
            PATH_OUT_TMPFILES = PATH_OUT_TMPFILES_PONLY
        }else if(PRED_MODE == "sexagebmi"){
            PATH_OUT_MODEL = PATH_OUT_MODEL_sexagebmi
            PATH_OUT_TMPFILES = PATH_OUT_TMPFILES_sexagebmi
        }else if(PRED_MODE == "comb"){
            PATH_OUT_MODEL = PATH_OUT_MODEL_COMB
            PATH_OUT_TMPFILES = PATH_OUT_TMPFILES_COMB
        }else{
            stop("No valid mode. set parameter to: prot_only or sexagebmi or comb")
        }

        # lsit of covariates that are potentially used as predictors
        if(PRED_MODE != "sexagebmi"){
            OUT_LABEL = paste0(SAMPLING, "_", "top", NR_PROTEINS, "_", label_rfe)
        }else{
            OUT_LABEL = paste0(SAMPLING, "_", PRED_MODE)
        }
        # /lustre/groups/itg/teams/zeggini/projects/fungen-oa/analyses/olink/classifier/ukbb_3000_oatraits//TKR/2_all_strict/models_nrfeat30/models/
        ##################################################################################
        # load input data (traintest/validation) for proteins and pheno
        get_matrices_obj = get_matrices(in_path_cc = PATH_CC, in_relcol_sub = IN_RELCOL_SUB, in_outcome = OA_PHENO, 
                                    in_path_sample_split = paste0(PATH_IN_ORIG, "sample_split_obj_COMBINED_", IN_DIR_LABEL, "_split91.rda"), 
                                    in_pred_mode = PRED_MODE)

        p_mtx_traintest_ukbb = get_matrices_obj$p_mtx_traintest
        pheno_train_test_ukbb = get_matrices_obj$pheno_train_test

        p_mtx_val_ukbb = get_matrices_obj$p_mtx_val
        pheno_val_ukbb = get_matrices_obj$pheno_val

        rm(get_matrices_obj)
        ##################################################################################
        # load input data (all); used for pwas
        # get_matrices_obj = get_matrices_allsamples(
        #     in_path_datalist = paste0(PATH_OUT_INIT,  "data_list_controlfromcontrolcolumn_COMBINED_", IN_DIR_LABEL, "_V9.rda"), 
        #                     in_relcol_sub = IN_RELCOL_SUB, is_bmi_used = (PRED_MODE != "prot_only"),oa_pheno_here = OA_PHENO, path_cc_here = PATH_CC)

        # pheno_all = get_matrices_obj$pheno_all
        # p_mtx_all = get_matrices_obj$p_mtx_all
        # identical(rownames(pheno_all), rownames(p_mtx_all))

        # rm(get_matrices_obj)
        ##################################################################################
        # PERFORM PWAS (in train test)
        if((PRED_MODE != "sexagebmi")){
            prot_sig = perform_pwas(in_p_mtx = p_mtx_traintest_ukbb[,!(colnames(p_mtx_traintest_ukbb) %in% PRED_COVS)], in_pheno = pheno_train_test_ukbb)
            tmp = prot_sig$pwas_sumstats
            save(tmp, file = paste0(PATH_OUT_MODEL, "pwas.rda"))
            rm(tmp)
            prot_sig = prot_sig$prot_sig

            # force inclusion of bmi, age and sex as predictors
            if((PRED_MODE != "prot_only")){
                prot_sig = c(prot_sig, PRED_COVS)
            }
        }
        ####################################################################################################################
        ####################################################################################################################
        # [HELPER] perform training testing
        # NOTE: 
        if((PRED_MODE != "sexagebmi")){
            if(PERFORM_TRAIN){
                ####################################################################################################################
                # clean tmp folder
                # clean_tmp_train_folder(in_p = PATH_OUT_TMPFILES)
                ####################################################################################################################
                # define data for script below
                p_in = pheno_train_test_ukbb
                m_in = p_mtx_traintest_ukbb[, prot_sig]
                ####################################################################################################################
                print(paste0(" --> train data samples in right order: ", identical(rownames(p_in), rownames(m_in))))
                print(paste0(" --> nr considered proteins: ", length(unique(colnames(m_in)))))
                ####################################################################################################################
                # define mtry around default value (sqrt(number features)); https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6198850/ --> corrected impurity is good
                tuneGrid = define_train_grid(nr_feat = length(prot_sig), rel_row = NULL) 

                # define training crtl
                if(SAMPLING != "none"){
                print("sampling method chosen")
                ctrl <- trainControl(method = "cv", number = 5, verboseIter = T, sampling = SAMPLING, 
                            allowParallel = TRUE)
                }else{
                ctrl <- trainControl(method = "cv", number = 5, verboseIter = T)
                }
                ####################################################################################################################
                # print(paste0("class of ctrl: ", ctrl))
                training_obj = perform_rf_training(in_number_iter = TRAIN_FI_RF_NR_ITER, 
                                in_number_folds = TRAIN_FI_RF_NR_ITER, m_in = m_in, 
                             p_in = p_in, in_ctrl = ctrl, in_grid = tuneGrid, outpath = PATH_OUT_TMPFILES)
                ##################################################################################            
            }
            ##################################################################################
            print(get_sen_spec_acc(in_p = PATH_OUT_TMPFILES))
            ##################################################################################
            # [OPTION 1]
            prot_mean = get_feat_importance(in_p = PATH_OUT_TMPFILES, in_mtx = t(p_mtx_traintest_ukbb[,prot_sig]), 
                        in_p_anno = get_p_anno(in_p = PATH_ANNO))
            print(paste0(cor.test(prot_mean$imp_mean, prot_mean$imp_median, method = "pearson")))

            # load pwas, merge in importance and write it out again
            tmp = get(load(file = paste0(PATH_OUT_MODEL, "pwas.rda")))
            prot_mean = cbind(prot_mean, tmp[rownames(prot_mean),])
            fwrite(prot_mean, file = paste0(PATH_OUT_MODEL, "pwas_nrfeat_", OUT_LABEL, ".txt.gz"), sep = "\t", compress = "gzip")
            # prot_mean = fread(file = paste0(PATH_OUT_MODEL, "pwas_nrfeat_", OUT_LABEL, ".txt.gz"), sep = "\t")

            # PLOT
            pdf(file = paste0(PATH_OUT_MODEL, "prot_importance_mean_CLEAN_", OUT_LABEL, ".pdf"), onefile = T)
            ggplot(prot_mean, aes(x=rank, y=imp_mean, color = "black")) + geom_point(show.legend = FALSE) + theme_light() +
                ylab("Mean feature\nimportance") + xlab("Proteins")
            ggplot(prot_mean[1:30,], aes(x=rank, y=imp_mean, color = "black")) + geom_point(show.legend = FALSE) + theme_light() +
                ylab("Mean feature\nimportance") + xlab("Proteins") 
            dev.off()
        }
        ###################################################################################################################
        # CHOSE FEATURES --> output: rel_features
        ##################################################################################
        # PRED_MODE --> prot_only|sexagebmi|comb
        if(PRED_MODE == "prot_only"){
            if(PERFORM_RFE){


                #in_pheno = pheno_train_test_ukbb
                #in_m = p_mtx_traintest_ukbb
                #in_dat = prot_mean
                #in_nr_proteins = NR_PROTEINS
                #rfe_number = 5
                #in_label = OUT_LABEL
                #in_p = PATH_OUT_MODEL


                # perform rfe 
                registerDoParallel(5)
                rel_features = run_rfe(in_pheno = pheno_train_test_ukbb, in_m = p_mtx_traintest_ukbb, in_dat = prot_mean, in_nr_proteins = NR_PROTEINS, 
                                            rfe_number = 5, in_label = OUT_LABEL, in_p = PATH_OUT_MODEL)
                
                # adjust label
                OUT_LABEL = rel_features$in_label

                # grep features
                rel_features = rel_features$rel_features
            }else{
                # grep features
                rel_features = getfrom_ranked_proteins()
            }
        }

        if(PRED_MODE == "sexagebmi"){
                rel_features = PRED_COVS
        }

        if(PRED_MODE == "comb"){
            if(PERFORM_RFE){
                # perform rfe (rfe potentially with age sex and bmi; if not, inclusion forced later.....; due to presence of age, sex and bmi, prot set of comb and prot_only 
                # can be different)

                # perform rfe for some comb cases
                if(adjust_comb_nrprots){
                    rfe_nr_prots = NR_PROTEINS_COMB
                }else{
                    rfe_nr_prots = NR_PROTEINS
                }


                registerDoParallel(5)

                # to be safe that "comb" does not include age, sex or bmi if they are not supposed to be there
                prot_mean = prot_mean[!(rownames(prot_mean) %in% PRED_COVS_NOTINCLUDE),]

                rel_features = run_rfe(in_pheno = pheno_train_test_ukbb, in_m = p_mtx_traintest_ukbb, in_dat = prot_mean, 
                    in_nr_proteins = rfe_nr_prots, rfe_number = 5, 
                    in_label = OUT_LABEL, in_p = PATH_OUT_MODEL)
                # rel_features = get(load(file = paste0(PATH_OUT_MODEL, "res_obj_feat.rda")))
                
                # adjust label
                OUT_LABEL = rel_features$in_label

                # grep features
                rel_features = rel_features$rel_features

                # force inclusion of sex, age and bmi
                rel_features = unique(c(rel_features, PRED_COVS))
            }else{
                # grep features
                rel_features = getfrom_ranked_proteins()

                # force inclusion of sex, age and bmi
                rel_features = unique(c(rel_features, PRED_COVS))
            }
        }
        ##################################################################################
        train_d = p_mtx_traintest_ukbb[,rel_features]
        dim(train_d)
        #  45541    50

        all(rel_features %in% colnames(train_d)) # T
        table(pheno_train_test_ukbb$oa_status)

        # make sure pheno and data matrix are in same order
        identical(rownames(train_d), rownames(pheno_train_test_ukbb)) # T
        ##################################################################################
        # define training
        set.seed(13)
        if(SAMPLING != "none"){
            print(paste("sampling ", SAMPLING,  " method chosen"))
            ctrl <- trainControl(method = "cv", number = 5, verboseIter = T, sampling = SAMPLING)
            }else{
            ctrl <- trainControl(method = "cv", number = 5, verboseIter = T)
        }
        ##################################################################################
        if(VAL_RF){
            print("Training RF")
            # build a random forest model
            # define mtry around default value (sqrt(number features)); https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6198850/ --> corrected impurity is good
            mtry = ceiling(sqrt(ncol(train_d)))
            mtry = unique(ceiling(c(mtry*0.6, mtry*0.8, mtry*1, mtry*1.2)))

            tuneGrid = data.frame(mtry = mtry, min.node.size = 1, splitrule = "gini")
            #  mtry min.node.size splitrule
            #1    5             1      gini
            #2    7             1      gini
            #3    8             1      gini
            #4   10             1      gini
            tuneGrid = data.frame(mtry = tuneGrid[, c("mtry")])

            TUNELENGTH_RF = nrow(tuneGrid)
            RF_MODEL_LABEL = paste0("model_rf_", OUT_LABEL, "_tunegrid", TUNELENGTH_RF , ".rda")

            log_file = paste0(strsplit(basename(RF_MODEL_LABEL), "\\.")[[1]][1], ".log")
            file.remove(paste0(PATH_OUT_MODEL, log_file))
            write_log_file(in_pheno_tt = pheno_train_test_ukbb, in_pheno_val = pheno_val_ukbb, 
                            in_feat = rel_features, in_log_file = log_file, in_prot_mean = prot_mean, 
                            in_dir = PATH_OUT_MODEL, in_pred_mode = PRED_MODE)

            model_rf <- caret::train(y = pheno_train_test_ukbb$oa_status, 
                                    # x = train_d, method = "ranger", 
                                    x = train_d, method = "rf", 
                                    preProcess = NULL, trControl = ctrl, metric = "Accuracy", 
                                    maximize = T, tuneGrid = tuneGrid, verbose = F)

            save(model_rf, file = paste0(PATH_OUT_MODEL, RF_MODEL_LABEL))   
            # /lustre/groups/itg/teams/zeggini/projects/fungen-oa/analyses/olink/classifier/ukbb_3000_oatraits//Combined_TKRKnee/2_all_strict//modelsV2/model_rf_rose_top50_norfe.rda
        }
        #/lustre/groups/itg/teams/zeggini/projects/fungen-oa/analyses/olink/classifier/ukbb_3000_oatraits//Knee/2_all_strict/models_nrfeat30/models/
        ##################################################################################
        # build an elastic net
        if(VAL_GLMNET){
            print("Training GLMNET")
            identical(rownames(train_d), rownames(pheno_train_test_ukbb)) # T

            GLMNET_MODEL_LABEL = paste0(PATH_OUT_MODEL, "model_glmnet_", OUT_LABEL, "_tunegrid", TUNELENGTH_GLMNET , ".rda")

            log_file = paste0(strsplit(basename(GLMNET_MODEL_LABEL), "\\.")[[1]][1], ".log")
            file.remove(paste0(PATH_OUT_MODEL, log_file))
            write_log_file(in_pheno_tt = pheno_train_test_ukbb, in_pheno_val = pheno_val_ukbb, in_feat = rel_features, in_log_file = log_file, in_prot_mean = prot_mean, 
                        in_dir = PATH_OUT_MODEL, in_pred_mode = PRED_MODE)

            model_glmnet <- caret::train(y = pheno_train_test_ukbb$oa_status, x = train_d, 
                                            method = "glmnet", preProcess = NULL, 
                                            trControl = ctrl, metric = "Accuracy", maximize = T, 
                                            tuneLength = TUNELENGTH_GLMNET, verbose = F, 
                                            importance  = "impurity_corrected")

            
            save(model_glmnet, file = paste0(PATH_OUT_MODEL, basename(GLMNET_MODEL_LABEL)))    
        }
        ##################################################################################
        if(VAL_XG){
            print("Training XG")
            registerDoParallel(1)

            # build complicated xgboost model; this is an advenced ensembl technique
            # this tutorial: https://odsc.medium.com/getting-up-to-speed-with-xgboost-in-r-ffbe402263f5
            identical(rownames(train_d), rownames(pheno_train_test_ukbb)) # T

            XG_MODEL_LABEL = paste0(PATH_OUT_MODEL, "model_xgboost_", OUT_LABEL, "_tunegrid", TUNELENGTH_XG , ".rda")

            log_file = paste0(strsplit(basename(XG_MODEL_LABEL), "\\.")[[1]][1], ".log")
            file.remove(paste0(PATH_OUT_MODEL, log_file))
            write_log_file(in_pheno_tt = pheno_train_test_ukbb, in_pheno_val = pheno_val_ukbb, in_feat = rel_features, in_log_file = log_file, in_prot_mean = prot_mean, 
                        in_dir = PATH_OUT_MODEL, in_pred_mode = PRED_MODE)

            model_xgboost <- caret::train(y = pheno_train_test_ukbb$oa_status, x = train_d, 
                                            method = "xgbTree", preProcess = NULL, 
                                            trControl = ctrl, metric = "Accuracy", maximize = T, 
                                            tuneLength = TUNELENGTH_XG, verbose = F, allowParallel = TRUE, num.threads = 5)

            
            save(model_xgboost, file = paste0(PATH_OUT_MODEL, basename(XG_MODEL_LABEL)))                                       
            # Selecting tuning parameters Fitting nrounds = 200, max_depth = 2, eta = 0.3, gamma = 0, 
            # colsample_bytree = 0.8, min_child_weight = 1, 
            # subsample = 0.875 on full training set
        }
        return(T)
    })
})
end_time <- Sys.time()
end_time - start_time
# Time difference of 2.050834 hours
####################################################################################################################################################################